# Zillow Database Construction Guide

#### In this guide I will demonstrate how to construct a zillow database, join zillow datasets, and more using terminal and sqlite. I will be working with California data.

## Terminal Lines

1) Open terminal and go to your scratch account: 

    cd /scratch/public/account_name
    
2) Move the Zillow data that you need in /scratch/public/ztrax to your directory /scratch/public/account_name

3) Optional: Use this to reduce the size of your zillow data based on your needs

    head -n zillow_data.txt > your_file.txt

4) Create your sqlite database:

    sqlite3 db_name.sqlite

### SQLite Lines

1) Create your SQL table. Check my github file SQLAlchemy.sqlite for a function to make these scripts:

> CREATE TABLE sm_trans_main_v1( TransId bigint(8), FIPS3 char(5), State3 char(2), County3 varchar(50), DataClassStndCode char(1), RecordTypeStndCode char(1), RecordingDate date(3), RecordingDocumentNumber varchar(25), RecordingBookNumber varchar(10), RecordingPageNumber varchar(10), ReRecordedCorrectionStndCode char(1), PriorRecordingDate date(3), PriorDocumentDate date(3), PriorDocumentNumber varchar(25), PriorBookNumber varchar(10), PriorPageNumber varchar(10), DocumentTypeStndCode char(4), DocumentDate date(3), SignatureDate date(3), EffectiveDate date(3), BuyerVestingStndCode char(2), BuyerMultiVestingFlag char(1), PartialInterestTransferStndCode char(1), PartialInterestTransferPercent decimal(5,3), SalesPriceAmount money(8), SalesPriceAmountStndCode char(2), CityTransferTax money(8), County3TransferTax money(8), State3TransferTax money(8), TotalTransferTax money(8), IntraFamilyTransferFlag char(1), TransferTaxExemptFlag char(1), PropertyUseStndCode char(2), AssessmentLandUseStndCode varchar(5), OccupancyStatusStndCode char(1), LegalStndCode char(1), BorrowerVestingStndCode char(2), LenderName varchar(250), LenderTypeStndCode char(2), LenderIDStndCode varchar(6), LenderDBAName varchar(135), DBALenderTypeStndCode char(2), DBALenderIDStndCode varchar(6), LenderMailCareOfName varchar(250), LenderMailHouseNumber varchar(13), LenderMailHouseNumberExt varchar(10), LenderMailStreetPreDirectional char(2), LenderMailStreetName varchar(50), LenderMailStreetSuffix varchar(6), LenderMailStreetPostDirectional char(2), LenderMailFullStreetAddress varchar(80), LenderMailBuildingName varchar(45), LenderMailBuildingNumber varchar(15), LenderMailUnitDesignator varchar(10), LenderMailUnit varchar(15), LenderMailCity varchar(45), LenderMailState3 char(2), LenderMailZip varchar(5), LenderMailZip4 char(4), LoanAmount money(8), LoanAmountStndCode char(1), MaximumLoanAmount money(8), LoanTypeStndCode char(2), LoanTypeClosedOpenEndStndCode char(1), LoanTypeFutureAdvanceFlag char(1), LoanTypeProgramStndCode char(2), LoanRateTypeStndCode char(3), LoanDueDate date(3), LoanTermMonths int(4), LoanTermYears int(4), InitialInterestRate decimal(5,3), ARMFirstAdjustmentDate date(3), ARMFirstAdjustmentMaxRate decimal(5,3), ARMFirstAdjustmentMinRate decimal(5,3), ARMIndexStndCode varchar(5), ARMAdjustmentFrequencyStndCode char(1), ARMMargin decimal(5,3), ARMInitialCap decimal(5,3), ARMPeriodicCap decimal(5,3), ARMLifetimeCap decimal(5,3), ARMMaxInterestRate decimal(5,3), ARMMinInterestRate decimal(5,3), InterestOnlyFlag char(1), InterestOnlyTerm char(3), PrepaymentPenaltyFlag char(1), PrepaymentPenaltyTerm char(3), BiWeeklyPaymentFlag char(1), AssumabilityRiderFlag char(1), BalloonRiderFlag char(1), CondominiumRiderFlag char(1), PlannedUnitDevelopmentRiderFlag char(1), SecondHomeRiderFlag char(1), OneToFourFamilyRiderFlag char(1), ConcurrentMtgeDocOrBkPg varchar(25), LoanNumber varchar(45), MERSMINNumber varchar(25), CaseNumber varchar(45), MERSFlag char(1), TitleCompanyName varchar(250), TitleCompanyIDStndCode varchar(6), AccommodationRecordingFlag char(1), UnpaidBalance money(8), InstallmentAmount money(8), InstallmentDueDate date(3), TotalDelinquentAmount money(8), DelinquentAsOfDate date(3), CurrentLender varchar(250), CurrentLenderTypeStndCode char(2), CurrentLenderIDStndCode varchar(6), TrusteeSaleNumber varchar(25), AttorneyFileNumber varchar(25), AuctionDate date(3), AuctionTime time(5), AuctionFullStreetAddress varchar(80), AuctionCityName varchar(45), StartingBid money(8), KeyedDate date(3), KeyerID varchar(10), SubVendorStndCode varchar(5), ImageFileName varchar(250), BuilderFlag char(1), MatchStndCode char(2), REOStndCode varchar(6), UpdateOwnershipFlag char(1), LoadID3 bigint(8), StatusInd int(4), TransactionTypeStndCode char(1), BatchID3 int(4), BKFSPID int(4), ZVendorStndCode char(3), SourceChkSum int(4));

2) Read in your zillow text file to your table:

> .separator '|'

> .import zillow_data.txt zillow_table

3) You can check the counties in your tables using:

    select distinct County3 from sm_trans_main_v1;

4) Optional: Get the county you need

    create table sm_trans_main_v2 as select * from sm_trans_main_v1 where County3 = 'SAN MATEO';





### Joining Tables

In this example I join multiple datasets. I add _P in order avoid name conflicts

5) Creating another table:

> CREATE TABLE trans_prop( TransId_P bigint(8), AssessorParcelNumber_P varchar(50), APNIndicatorStndCode_P char(1), TaxIDNumber_P varchar(50), TaxIDIndicatorStndCode_P char(1), UnformattedAssessorParcelNumber_P varchar(50), AlternateParcelNumber_P varchar(50), HawaiiCondoCPRCode_P char(4), PropertyHouseNumber_P varchar(13), PropertyHouseNumberExt_P varchar(10), PropertyStreetPreDirectional_P char(2), PropertyStreetName_P varchar(50), PropertyStreetSuffix_P varchar(6), PropertyStreetPostDirectional_P char(2), PropertyBuildingNumber_P varchar(45), PropertyFullStreetAddress_P varchar(80), PropertyCity_P varchar(45), PropertyState_P char(2), PropertyZip_P varchar(5), PropertyZip4_P char(4), OriginalPropertyFullStreetAddress_P varchar(100), OriginalPropertyAddressLastline_P varchar(100), PropertyAddressStndCode_P char(1), LegalLot_P varchar(100), LegalOtherLot_P varchar(100), LegalLotCode_P char(2), LegalBlock_P varchar(50), LegalSubdivisionName_P varchar(200), LegalCondoProjectPUDDevName_P varchar(100), LegalBuildingNumber_P varchar(45), LegalUnit_P varchar(100), LegalSection_P varchar(100), LegalPhase_P varchar(50), LegalTract_P varchar(50), LegalDistrict_P varchar(45), LegalMunicipality_P varchar(50), LegalCity_P varchar(50), LegalTownship_P varchar(50), LegalSTRSection_P varchar(100), LegalSTRTownship_P varchar(15), LegalSTRRange_P varchar(15), LegalSTRMeridian_P varchar(35), LegalSecTwnRngMer_P varchar(165), LegalRecordersMapReference_P varchar(100), LegalDescription_P varchar(2000), LegalLotSize_P varchar(14), PropertySequenceNumber_P int(4), PropertyAddressMatchcode_P char(1), PropertyAddressUnitDesignator_P varchar(10), PropertyAddressUnitNumber_P varchar(11), PropertyAddressCarrierRoute_P char(4), PropertyAddressGeoCodeMatchCode_P char(1), PropertyAddressLatitude_P decimal(9,6), PropertyAddressLongitude_P decimal(9,6), PropertyAddressCensusTractAndBlock_P varchar(16), PropertyAddressConfidenceScore_P tinyint(1), PropertyAddressCBSACode_P int(4), PropertyAddressCBSADivisionCode_P int(4), PropertyAddressMatchType_P tinyint(1), PropertyAddressDPV_P char(1), PropertyGeocodeQualityCode_P varchar(10), PropertyAddressQualityCode_P varchar(10), FIPS_P char(5), LoadID_P bigint(8), ImportParcelID_P bigint(8), BKFSPID_P int(4), AssessmentRecordMatchFlag_P tinyint(1), BatchID_P int(4));

6) Joining trans_prop and sm_trans_main_v2
> create table sm_trans_main_prop as select * from sm_trans_main_v2 inner join trans_prop on trans_prop.TransId_P = sm_trans_main_v2.TransId;


7) Attaching Assessor Main for SM
>  attach 'ZMainBldgCA.sqlite' as zmb;

8) Join SM Assessor Main with SM Trans Main and Property Info
> create table sm_trans_prop_assr as select * from sm_trans_main_prop inner join zmb.san_mateo on sm_trans_main_prop.ImportParcelID_P = zmb.san_mateo.ImportParcelID;

9) Convert to csv

> .header on

> .mode csv

> .once /scratch/public/account/sm_trans_prop_assr.csv

> select * from sm_trans_prop_assr;



### Static Pivot Query
> select t.TransId, max(case when seqnum = 1 then FIPS3 end) as FIPS3_1, max(case when seqnum = 2 then FIPS3 end) as FIPS3_2, max(case when seqnum = 3 then FIPS3 end) as FIPS3_3, max(case when seqnum = 4 then FIPS3 end) as FIPS3_4, max(case when seqnum = 5 then FIPS3 end) as FIPS3_5, max(case when seqnum = 1 then State3 end) as State3_1, max(case when seqnum = 2 then State3 end) as State3_2, max(case when seqnum = 3 then State3 end) as State3_3, max(case when seqnum = 4 then State3 end) as State3_4, max(case when seqnum = 5 then State3 end) as State3_5, max(case when seqnum = 1 then County3 end) as County3_1, max(case when seqnum = 2 then County3 end) as County3_2, max(case when seqnum = 3 then County3 end) as County3_3, max(case when seqnum = 4 then County3 end) as County3_4, max(case when seqnum = 5 then County3 end) as County3_5, max(case when seqnum = 1 then DataClassStndCode end) as DataClassStndCode_1, max(case when seqnum = 2 then DataClassStndCode end) as DataClassStndCode_2, max(case when seqnum = 3 then DataClassStndCode end) as DataClassStndCode_3, max(case when seqnum = 4 then DataClassStndCode end) as DataClassStndCode_4, max(case when seqnum = 5 then DataClassStndCode end) as DataClassStndCode_5, max(case when seqnum = 1 then RecordTypeStndCode end) as RecordTypeStndCode_1, max(case when seqnum = 2 then RecordTypeStndCode end) as RecordTypeStndCode_2, max(case when seqnum = 3 then RecordTypeStndCode end) as RecordTypeStndCode_3, max(case when seqnum = 4 then RecordTypeStndCode end) as RecordTypeStndCode_4, max(case when seqnum = 5 then RecordTypeStndCode end) as RecordTypeStndCode_5, max(case when seqnum = 1 then RecordingDate end) as RecordingDate_1, max(case when seqnum = 2 then RecordingDate end) as RecordingDate_2, max(case when seqnum = 3 then RecordingDate end) as RecordingDate_3, max(case when seqnum = 4 then RecordingDate end) as RecordingDate_4, max(case when seqnum = 5 then RecordingDate end) as RecordingDate_5, max(case when seqnum = 1 then RecordingDocumentNumber end) as RecordingDocumentNumber_1, max(case when seqnum = 2 then RecordingDocumentNumber end) as RecordingDocumentNumber_2, max(case when seqnum = 3 then RecordingDocumentNumber end) as RecordingDocumentNumber_3, max(case when seqnum = 4 then RecordingDocumentNumber end) as RecordingDocumentNumber_4, max(case when seqnum = 5 then RecordingDocumentNumber end) as RecordingDocumentNumber_5, max(case when seqnum = 1 then RecordingBookNumber end) as RecordingBookNumber_1, max(case when seqnum = 2 then RecordingBookNumber end) as RecordingBookNumber_2, max(case when seqnum = 3 then RecordingBookNumber end) as RecordingBookNumber_3, max(case when seqnum = 4 then RecordingBookNumber end) as RecordingBookNumber_4, max(case when seqnum = 5 then RecordingBookNumber end) as RecordingBookNumber_5, max(case when seqnum = 1 then RecordingPageNumber end) as RecordingPageNumber_1, max(case when seqnum = 2 then RecordingPageNumber end) as RecordingPageNumber_2, max(case when seqnum = 3 then RecordingPageNumber end) as RecordingPageNumber_3, max(case when seqnum = 4 then RecordingPageNumber end) as RecordingPageNumber_4, max(case when seqnum = 5 then RecordingPageNumber end) as RecordingPageNumber_5, max(case when seqnum = 1 then ReRecordedCorrectionStndCode end) as ReRecordedCorrectionStndCode_1, max(case when seqnum = 2 then ReRecordedCorrectionStndCode end) as ReRecordedCorrectionStndCode_2, max(case when seqnum = 3 then ReRecordedCorrectionStndCode end) as ReRecordedCorrectionStndCode_3, max(case when seqnum = 4 then ReRecordedCorrectionStndCode end) as ReRecordedCorrectionStndCode_4, max(case when seqnum = 5 then ReRecordedCorrectionStndCode end) as ReRecordedCorrectionStndCode_5, max(case when seqnum = 1 then PriorRecordingDate end) as PriorRecordingDate_1, max(case when seqnum = 2 then PriorRecordingDate end) as PriorRecordingDate_2, max(case when seqnum = 3 then PriorRecordingDate end) as PriorRecordingDate_3, max(case when seqnum = 4 then PriorRecordingDate end) as PriorRecordingDate_4, max(case when seqnum = 5 then PriorRecordingDate end) as PriorRecordingDate_5, max(case when seqnum = 1 then PriorDocumentDate end) as PriorDocumentDate_1, max(case when seqnum = 2 then PriorDocumentDate end) as PriorDocumentDate_2, max(case when seqnum = 3 then PriorDocumentDate end) as PriorDocumentDate_3, max(case when seqnum = 4 then PriorDocumentDate end) as PriorDocumentDate_4, max(case when seqnum = 5 then PriorDocumentDate end) as PriorDocumentDate_5, max(case when seqnum = 1 then PriorDocumentNumber end) as PriorDocumentNumber_1, max(case when seqnum = 2 then PriorDocumentNumber end) as PriorDocumentNumber_2, max(case when seqnum = 3 then PriorDocumentNumber end) as PriorDocumentNumber_3, max(case when seqnum = 4 then PriorDocumentNumber end) as PriorDocumentNumber_4, max(case when seqnum = 5 then PriorDocumentNumber end) as PriorDocumentNumber_5, max(case when seqnum = 1 then PriorBookNumber end) as PriorBookNumber_1, max(case when seqnum = 2 then PriorBookNumber end) as PriorBookNumber_2, max(case when seqnum = 3 then PriorBookNumber end) as PriorBookNumber_3, max(case when seqnum = 4 then PriorBookNumber end) as PriorBookNumber_4, max(case when seqnum = 5 then PriorBookNumber end) as PriorBookNumber_5, max(case when seqnum = 1 then PriorPageNumber end) as PriorPageNumber_1, max(case when seqnum = 2 then PriorPageNumber end) as PriorPageNumber_2, max(case when seqnum = 3 then PriorPageNumber end) as PriorPageNumber_3, max(case when seqnum = 4 then PriorPageNumber end) as PriorPageNumber_4, max(case when seqnum = 5 then PriorPageNumber end) as PriorPageNumber_5, max(case when seqnum = 1 then DocumentTypeStndCode end) as DocumentTypeStndCode_1, max(case when seqnum = 2 then DocumentTypeStndCode end) as DocumentTypeStndCode_2, max(case when seqnum = 3 then DocumentTypeStndCode end) as DocumentTypeStndCode_3, max(case when seqnum = 4 then DocumentTypeStndCode end) as DocumentTypeStndCode_4, max(case when seqnum = 5 then DocumentTypeStndCode end) as DocumentTypeStndCode_5, max(case when seqnum = 1 then DocumentDate end) as DocumentDate_1, max(case when seqnum = 2 then DocumentDate end) as DocumentDate_2, max(case when seqnum = 3 then DocumentDate end) as DocumentDate_3, max(case when seqnum = 4 then DocumentDate end) as DocumentDate_4, max(case when seqnum = 5 then DocumentDate end) as DocumentDate_5, max(case when seqnum = 1 then SignatureDate end) as SignatureDate_1, max(case when seqnum = 2 then SignatureDate end) as SignatureDate_2, max(case when seqnum = 3 then SignatureDate end) as SignatureDate_3, max(case when seqnum = 4 then SignatureDate end) as SignatureDate_4, max(case when seqnum = 5 then SignatureDate end) as SignatureDate_5, max(case when seqnum = 1 then EffectiveDate end) as EffectiveDate_1, max(case when seqnum = 2 then EffectiveDate end) as EffectiveDate_2, max(case when seqnum = 3 then EffectiveDate end) as EffectiveDate_3, max(case when seqnum = 4 then EffectiveDate end) as EffectiveDate_4, max(case when seqnum = 5 then EffectiveDate end) as EffectiveDate_5, max(case when seqnum = 1 then BuyerVestingStndCode end) as BuyerVestingStndCode_1, max(case when seqnum = 2 then BuyerVestingStndCode end) as BuyerVestingStndCode_2, max(case when seqnum = 3 then BuyerVestingStndCode end) as BuyerVestingStndCode_3, max(case when seqnum = 4 then BuyerVestingStndCode end) as BuyerVestingStndCode_4, max(case when seqnum = 5 then BuyerVestingStndCode end) as BuyerVestingStndCode_5, max(case when seqnum = 1 then BuyerMultiVestingFlag end) as BuyerMultiVestingFlag_1, max(case when seqnum = 2 then BuyerMultiVestingFlag end) as BuyerMultiVestingFlag_2, max(case when seqnum = 3 then BuyerMultiVestingFlag end) as BuyerMultiVestingFlag_3, max(case when seqnum = 4 then BuyerMultiVestingFlag end) as BuyerMultiVestingFlag_4, max(case when seqnum = 5 then BuyerMultiVestingFlag end) as BuyerMultiVestingFlag_5, max(case when seqnum = 1 then PartialInterestTransferStndCode end) as PartialInterestTransferStndCode_1, max(case when seqnum = 2 then PartialInterestTransferStndCode end) as PartialInterestTransferStndCode_2, max(case when seqnum = 3 then PartialInterestTransferStndCode end) as PartialInterestTransferStndCode_3, max(case when seqnum = 4 then PartialInterestTransferStndCode end) as PartialInterestTransferStndCode_4, max(case when seqnum = 5 then PartialInterestTransferStndCode end) as PartialInterestTransferStndCode_5, max(case when seqnum = 1 then PartialInterestTransferPercent end) as PartialInterestTransferPercent_1, max(case when seqnum = 2 then PartialInterestTransferPercent end) as PartialInterestTransferPercent_2, max(case when seqnum = 3 then PartialInterestTransferPercent end) as PartialInterestTransferPercent_3, max(case when seqnum = 4 then PartialInterestTransferPercent end) as PartialInterestTransferPercent_4, max(case when seqnum = 5 then PartialInterestTransferPercent end) as PartialInterestTransferPercent_5, max(case when seqnum = 1 then SalesPriceAmount end) as SalesPriceAmount_1, max(case when seqnum = 2 then SalesPriceAmount end) as SalesPriceAmount_2, max(case when seqnum = 3 then SalesPriceAmount end) as SalesPriceAmount_3, max(case when seqnum = 4 then SalesPriceAmount end) as SalesPriceAmount_4, max(case when seqnum = 5 then SalesPriceAmount end) as SalesPriceAmount_5, max(case when seqnum = 1 then SalesPriceAmountStndCode end) as SalesPriceAmountStndCode_1, max(case when seqnum = 2 then SalesPriceAmountStndCode end) as SalesPriceAmountStndCode_2, max(case when seqnum = 3 then SalesPriceAmountStndCode end) as SalesPriceAmountStndCode_3, max(case when seqnum = 4 then SalesPriceAmountStndCode end) as SalesPriceAmountStndCode_4, max(case when seqnum = 5 then SalesPriceAmountStndCode end) as SalesPriceAmountStndCode_5, max(case when seqnum = 1 then CityTransferTax end) as CityTransferTax_1, max(case when seqnum = 2 then CityTransferTax end) as CityTransferTax_2, max(case when seqnum = 3 then CityTransferTax end) as CityTransferTax_3, max(case when seqnum = 4 then CityTransferTax end) as CityTransferTax_4, max(case when seqnum = 5 then CityTransferTax end) as CityTransferTax_5, max(case when seqnum = 1 then County3TransferTax end) as County3TransferTax_1, max(case when seqnum = 2 then County3TransferTax end) as County3TransferTax_2, max(case when seqnum = 3 then County3TransferTax end) as County3TransferTax_3, max(case when seqnum = 4 then County3TransferTax end) as County3TransferTax_4, max(case when seqnum = 5 then County3TransferTax end) as County3TransferTax_5, max(case when seqnum = 1 then State3TransferTax end) as State3TransferTax_1, max(case when seqnum = 2 then State3TransferTax end) as State3TransferTax_2, max(case when seqnum = 3 then State3TransferTax end) as State3TransferTax_3, max(case when seqnum = 4 then State3TransferTax end) as State3TransferTax_4, max(case when seqnum = 5 then State3TransferTax end) as State3TransferTax_5, max(case when seqnum = 1 then TotalTransferTax end) as TotalTransferTax_1, max(case when seqnum = 2 then TotalTransferTax end) as TotalTransferTax_2, max(case when seqnum = 3 then TotalTransferTax end) as TotalTransferTax_3, max(case when seqnum = 4 then TotalTransferTax end) as TotalTransferTax_4, max(case when seqnum = 5 then TotalTransferTax end) as TotalTransferTax_5, max(case when seqnum = 1 then IntraFamilyTransferFlag end) as IntraFamilyTransferFlag_1, max(case when seqnum = 2 then IntraFamilyTransferFlag end) as IntraFamilyTransferFlag_2, max(case when seqnum = 3 then IntraFamilyTransferFlag end) as IntraFamilyTransferFlag_3, max(case when seqnum = 4 then IntraFamilyTransferFlag end) as IntraFamilyTransferFlag_4, max(case when seqnum = 5 then IntraFamilyTransferFlag end) as IntraFamilyTransferFlag_5, max(case when seqnum = 1 then TransferTaxExemptFlag end) as TransferTaxExemptFlag_1, max(case when seqnum = 2 then TransferTaxExemptFlag end) as TransferTaxExemptFlag_2, max(case when seqnum = 3 then TransferTaxExemptFlag end) as TransferTaxExemptFlag_3, max(case when seqnum = 4 then TransferTaxExemptFlag end) as TransferTaxExemptFlag_4, max(case when seqnum = 5 then TransferTaxExemptFlag end) as TransferTaxExemptFlag_5, max(case when seqnum = 1 then PropertyUseStndCode end) as PropertyUseStndCode_1, max(case when seqnum = 2 then PropertyUseStndCode end) as PropertyUseStndCode_2, max(case when seqnum = 3 then PropertyUseStndCode end) as PropertyUseStndCode_3, max(case when seqnum = 4 then PropertyUseStndCode end) as PropertyUseStndCode_4, max(case when seqnum = 5 then PropertyUseStndCode end) as PropertyUseStndCode_5, max(case when seqnum = 1 then AssessmentLandUseStndCode end) as AssessmentLandUseStndCode_1, max(case when seqnum = 2 then AssessmentLandUseStndCode end) as AssessmentLandUseStndCode_2, max(case when seqnum = 3 then AssessmentLandUseStndCode end) as AssessmentLandUseStndCode_3, max(case when seqnum = 4 then AssessmentLandUseStndCode end) as AssessmentLandUseStndCode_4, max(case when seqnum = 5 then AssessmentLandUseStndCode end) as AssessmentLandUseStndCode_5, max(case when seqnum = 1 then OccupancyStatusStndCode end) as OccupancyStatusStndCode_1, max(case when seqnum = 2 then OccupancyStatusStndCode end) as OccupancyStatusStndCode_2, max(case when seqnum = 3 then OccupancyStatusStndCode end) as OccupancyStatusStndCode_3, max(case when seqnum = 4 then OccupancyStatusStndCode end) as OccupancyStatusStndCode_4, max(case when seqnum = 5 then OccupancyStatusStndCode end) as OccupancyStatusStndCode_5, max(case when seqnum = 1 then LegalStndCode end) as LegalStndCode_1, max(case when seqnum = 2 then LegalStndCode end) as LegalStndCode_2, max(case when seqnum = 3 then LegalStndCode end) as LegalStndCode_3, max(case when seqnum = 4 then LegalStndCode end) as LegalStndCode_4, max(case when seqnum = 5 then LegalStndCode end) as LegalStndCode_5, max(case when seqnum = 1 then BorrowerVestingStndCode end) as BorrowerVestingStndCode_1, max(case when seqnum = 2 then BorrowerVestingStndCode end) as BorrowerVestingStndCode_2, max(case when seqnum = 3 then BorrowerVestingStndCode end) as BorrowerVestingStndCode_3, max(case when seqnum = 4 then BorrowerVestingStndCode end) as BorrowerVestingStndCode_4, max(case when seqnum = 5 then BorrowerVestingStndCode end) as BorrowerVestingStndCode_5, max(case when seqnum = 1 then LenderName end) as LenderName_1, max(case when seqnum = 2 then LenderName end) as LenderName_2, max(case when seqnum = 3 then LenderName end) as LenderName_3, max(case when seqnum = 4 then LenderName end) as LenderName_4, max(case when seqnum = 5 then LenderName end) as LenderName_5, max(case when seqnum = 1 then LenderTypeStndCode end) as LenderTypeStndCode_1, max(case when seqnum = 2 then LenderTypeStndCode end) as LenderTypeStndCode_2, max(case when seqnum = 3 then LenderTypeStndCode end) as LenderTypeStndCode_3, max(case when seqnum = 4 then LenderTypeStndCode end) as LenderTypeStndCode_4, max(case when seqnum = 5 then LenderTypeStndCode end) as LenderTypeStndCode_5, max(case when seqnum = 1 then LenderIDStndCode end) as LenderIDStndCode_1, max(case when seqnum = 2 then LenderIDStndCode end) as LenderIDStndCode_2, max(case when seqnum = 3 then LenderIDStndCode end) as LenderIDStndCode_3, max(case when seqnum = 4 then LenderIDStndCode end) as LenderIDStndCode_4, max(case when seqnum = 5 then LenderIDStndCode end) as LenderIDStndCode_5, max(case when seqnum = 1 then LenderDBAName end) as LenderDBAName_1, max(case when seqnum = 2 then LenderDBAName end) as LenderDBAName_2, max(case when seqnum = 3 then LenderDBAName end) as LenderDBAName_3, max(case when seqnum = 4 then LenderDBAName end) as LenderDBAName_4, max(case when seqnum = 5 then LenderDBAName end) as LenderDBAName_5, max(case when seqnum = 1 then DBALenderTypeStndCode end) as DBALenderTypeStndCode_1, max(case when seqnum = 2 then DBALenderTypeStndCode end) as DBALenderTypeStndCode_2, max(case when seqnum = 3 then DBALenderTypeStndCode end) as DBALenderTypeStndCode_3, max(case when seqnum = 4 then DBALenderTypeStndCode end) as DBALenderTypeStndCode_4, max(case when seqnum = 5 then DBALenderTypeStndCode end) as DBALenderTypeStndCode_5, max(case when seqnum = 1 then DBALenderIDStndCode end) as DBALenderIDStndCode_1, max(case when seqnum = 2 then DBALenderIDStndCode end) as DBALenderIDStndCode_2, max(case when seqnum = 3 then DBALenderIDStndCode end) as DBALenderIDStndCode_3, max(case when seqnum = 4 then DBALenderIDStndCode end) as DBALenderIDStndCode_4, max(case when seqnum = 5 then DBALenderIDStndCode end) as DBALenderIDStndCode_5, max(case when seqnum = 1 then LenderMailCareOfName end) as LenderMailCareOfName_1, max(case when seqnum = 2 then LenderMailCareOfName end) as LenderMailCareOfName_2, max(case when seqnum = 3 then LenderMailCareOfName end) as LenderMailCareOfName_3, max(case when seqnum = 4 then LenderMailCareOfName end) as LenderMailCareOfName_4, max(case when seqnum = 5 then LenderMailCareOfName end) as LenderMailCareOfName_5, max(case when seqnum = 1 then LenderMailHouseNumber end) as LenderMailHouseNumber_1, max(case when seqnum = 2 then LenderMailHouseNumber end) as LenderMailHouseNumber_2, max(case when seqnum = 3 then LenderMailHouseNumber end) as LenderMailHouseNumber_3, max(case when seqnum = 4 then LenderMailHouseNumber end) as LenderMailHouseNumber_4, max(case when seqnum = 5 then LenderMailHouseNumber end) as LenderMailHouseNumber_5, max(case when seqnum = 1 then LenderMailHouseNumberExt end) as LenderMailHouseNumberExt_1, max(case when seqnum = 2 then LenderMailHouseNumberExt end) as LenderMailHouseNumberExt_2, max(case when seqnum = 3 then LenderMailHouseNumberExt end) as LenderMailHouseNumberExt_3, max(case when seqnum = 4 then LenderMailHouseNumberExt end) as LenderMailHouseNumberExt_4, max(case when seqnum = 5 then LenderMailHouseNumberExt end) as LenderMailHouseNumberExt_5, max(case when seqnum = 1 then LenderMailStreetPreDirectional end) as LenderMailStreetPreDirectional_1, max(case when seqnum = 2 then LenderMailStreetPreDirectional end) as LenderMailStreetPreDirectional_2, max(case when seqnum = 3 then LenderMailStreetPreDirectional end) as LenderMailStreetPreDirectional_3, max(case when seqnum = 4 then LenderMailStreetPreDirectional end) as LenderMailStreetPreDirectional_4, max(case when seqnum = 5 then LenderMailStreetPreDirectional end) as LenderMailStreetPreDirectional_5, max(case when seqnum = 1 then LenderMailStreetName end) as LenderMailStreetName_1, max(case when seqnum = 2 then LenderMailStreetName end) as LenderMailStreetName_2, max(case when seqnum = 3 then LenderMailStreetName end) as LenderMailStreetName_3, max(case when seqnum = 4 then LenderMailStreetName end) as LenderMailStreetName_4, max(case when seqnum = 5 then LenderMailStreetName end) as LenderMailStreetName_5, max(case when seqnum = 1 then LenderMailStreetSuffix end) as LenderMailStreetSuffix_1, max(case when seqnum = 2 then LenderMailStreetSuffix end) as LenderMailStreetSuffix_2, max(case when seqnum = 3 then LenderMailStreetSuffix end) as LenderMailStreetSuffix_3, max(case when seqnum = 4 then LenderMailStreetSuffix end) as LenderMailStreetSuffix_4, max(case when seqnum = 5 then LenderMailStreetSuffix end) as LenderMailStreetSuffix_5, max(case when seqnum = 1 then LenderMailStreetPostDirectional end) as LenderMailStreetPostDirectional_1, max(case when seqnum = 2 then LenderMailStreetPostDirectional end) as LenderMailStreetPostDirectional_2, max(case when seqnum = 3 then LenderMailStreetPostDirectional end) as LenderMailStreetPostDirectional_3, max(case when seqnum = 4 then LenderMailStreetPostDirectional end) as LenderMailStreetPostDirectional_4, max(case when seqnum = 5 then LenderMailStreetPostDirectional end) as LenderMailStreetPostDirectional_5, max(case when seqnum = 1 then LenderMailFullStreetAddress end) as LenderMailFullStreetAddress_1, max(case when seqnum = 2 then LenderMailFullStreetAddress end) as LenderMailFullStreetAddress_2, max(case when seqnum = 3 then LenderMailFullStreetAddress end) as LenderMailFullStreetAddress_3, max(case when seqnum = 4 then LenderMailFullStreetAddress end) as LenderMailFullStreetAddress_4, max(case when seqnum = 5 then LenderMailFullStreetAddress end) as LenderMailFullStreetAddress_5, max(case when seqnum = 1 then LenderMailBuildingName end) as LenderMailBuildingName_1, max(case when seqnum = 2 then LenderMailBuildingName end) as LenderMailBuildingName_2, max(case when seqnum = 3 then LenderMailBuildingName end) as LenderMailBuildingName_3, max(case when seqnum = 4 then LenderMailBuildingName end) as LenderMailBuildingName_4, max(case when seqnum = 5 then LenderMailBuildingName end) as LenderMailBuildingName_5, max(case when seqnum = 1 then LenderMailBuildingNumber end) as LenderMailBuildingNumber_1, max(case when seqnum = 2 then LenderMailBuildingNumber end) as LenderMailBuildingNumber_2, max(case when seqnum = 3 then LenderMailBuildingNumber end) as LenderMailBuildingNumber_3, max(case when seqnum = 4 then LenderMailBuildingNumber end) as LenderMailBuildingNumber_4, max(case when seqnum = 5 then LenderMailBuildingNumber end) as LenderMailBuildingNumber_5, max(case when seqnum = 1 then LenderMailUnitDesignator end) as LenderMailUnitDesignator_1, max(case when seqnum = 2 then LenderMailUnitDesignator end) as LenderMailUnitDesignator_2, max(case when seqnum = 3 then LenderMailUnitDesignator end) as LenderMailUnitDesignator_3, max(case when seqnum = 4 then LenderMailUnitDesignator end) as LenderMailUnitDesignator_4, max(case when seqnum = 5 then LenderMailUnitDesignator end) as LenderMailUnitDesignator_5, max(case when seqnum = 1 then LenderMailUnit end) as LenderMailUnit_1, max(case when seqnum = 2 then LenderMailUnit end) as LenderMailUnit_2, max(case when seqnum = 3 then LenderMailUnit end) as LenderMailUnit_3, max(case when seqnum = 4 then LenderMailUnit end) as LenderMailUnit_4, max(case when seqnum = 5 then LenderMailUnit end) as LenderMailUnit_5, max(case when seqnum = 1 then LenderMailCity end) as LenderMailCity_1, max(case when seqnum = 2 then LenderMailCity end) as LenderMailCity_2, max(case when seqnum = 3 then LenderMailCity end) as LenderMailCity_3, max(case when seqnum = 4 then LenderMailCity end) as LenderMailCity_4, max(case when seqnum = 5 then LenderMailCity end) as LenderMailCity_5, max(case when seqnum = 1 then LenderMailState3 end) as LenderMailState3_1, max(case when seqnum = 2 then LenderMailState3 end) as LenderMailState3_2, max(case when seqnum = 3 then LenderMailState3 end) as LenderMailState3_3, max(case when seqnum = 4 then LenderMailState3 end) as LenderMailState3_4, max(case when seqnum = 5 then LenderMailState3 end) as LenderMailState3_5, max(case when seqnum = 1 then LenderMailZip end) as LenderMailZip_1, max(case when seqnum = 2 then LenderMailZip end) as LenderMailZip_2, max(case when seqnum = 3 then LenderMailZip end) as LenderMailZip_3, max(case when seqnum = 4 then LenderMailZip end) as LenderMailZip_4, max(case when seqnum = 5 then LenderMailZip end) as LenderMailZip_5, max(case when seqnum = 1 then LenderMailZip4 end) as LenderMailZip4_1, max(case when seqnum = 2 then LenderMailZip4 end) as LenderMailZip4_2, max(case when seqnum = 3 then LenderMailZip4 end) as LenderMailZip4_3, max(case when seqnum = 4 then LenderMailZip4 end) as LenderMailZip4_4, max(case when seqnum = 5 then LenderMailZip4 end) as LenderMailZip4_5, max(case when seqnum = 1 then LoanAmount end) as LoanAmount_1, max(case when seqnum = 2 then LoanAmount end) as LoanAmount_2, max(case when seqnum = 3 then LoanAmount end) as LoanAmount_3, max(case when seqnum = 4 then LoanAmount end) as LoanAmount_4, max(case when seqnum = 5 then LoanAmount end) as LoanAmount_5, max(case when seqnum = 1 then LoanAmountStndCode end) as LoanAmountStndCode_1, max(case when seqnum = 2 then LoanAmountStndCode end) as LoanAmountStndCode_2, max(case when seqnum = 3 then LoanAmountStndCode end) as LoanAmountStndCode_3, max(case when seqnum = 4 then LoanAmountStndCode end) as LoanAmountStndCode_4, max(case when seqnum = 5 then LoanAmountStndCode end) as LoanAmountStndCode_5, max(case when seqnum = 1 then MaximumLoanAmount end) as MaximumLoanAmount_1, max(case when seqnum = 2 then MaximumLoanAmount end) as MaximumLoanAmount_2, max(case when seqnum = 3 then MaximumLoanAmount end) as MaximumLoanAmount_3, max(case when seqnum = 4 then MaximumLoanAmount end) as MaximumLoanAmount_4, max(case when seqnum = 5 then MaximumLoanAmount end) as MaximumLoanAmount_5, max(case when seqnum = 1 then LoanTypeStndCode end) as LoanTypeStndCode_1, max(case when seqnum = 2 then LoanTypeStndCode end) as LoanTypeStndCode_2, max(case when seqnum = 3 then LoanTypeStndCode end) as LoanTypeStndCode_3, max(case when seqnum = 4 then LoanTypeStndCode end) as LoanTypeStndCode_4, max(case when seqnum = 5 then LoanTypeStndCode end) as LoanTypeStndCode_5, max(case when seqnum = 1 then LoanTypeClosedOpenEndStndCode end) as LoanTypeClosedOpenEndStndCode_1, max(case when seqnum = 2 then LoanTypeClosedOpenEndStndCode end) as LoanTypeClosedOpenEndStndCode_2, max(case when seqnum = 3 then LoanTypeClosedOpenEndStndCode end) as LoanTypeClosedOpenEndStndCode_3, max(case when seqnum = 4 then LoanTypeClosedOpenEndStndCode end) as LoanTypeClosedOpenEndStndCode_4, max(case when seqnum = 5 then LoanTypeClosedOpenEndStndCode end) as LoanTypeClosedOpenEndStndCode_5, max(case when seqnum = 1 then LoanTypeFutureAdvanceFlag end) as LoanTypeFutureAdvanceFlag_1, max(case when seqnum = 2 then LoanTypeFutureAdvanceFlag end) as LoanTypeFutureAdvanceFlag_2, max(case when seqnum = 3 then LoanTypeFutureAdvanceFlag end) as LoanTypeFutureAdvanceFlag_3, max(case when seqnum = 4 then LoanTypeFutureAdvanceFlag end) as LoanTypeFutureAdvanceFlag_4, max(case when seqnum = 5 then LoanTypeFutureAdvanceFlag end) as LoanTypeFutureAdvanceFlag_5, max(case when seqnum = 1 then LoanTypeProgramStndCode end) as LoanTypeProgramStndCode_1, max(case when seqnum = 2 then LoanTypeProgramStndCode end) as LoanTypeProgramStndCode_2, max(case when seqnum = 3 then LoanTypeProgramStndCode end) as LoanTypeProgramStndCode_3, max(case when seqnum = 4 then LoanTypeProgramStndCode end) as LoanTypeProgramStndCode_4, max(case when seqnum = 5 then LoanTypeProgramStndCode end) as LoanTypeProgramStndCode_5, max(case when seqnum = 1 then LoanRateTypeStndCode end) as LoanRateTypeStndCode_1, max(case when seqnum = 2 then LoanRateTypeStndCode end) as LoanRateTypeStndCode_2, max(case when seqnum = 3 then LoanRateTypeStndCode end) as LoanRateTypeStndCode_3, max(case when seqnum = 4 then LoanRateTypeStndCode end) as LoanRateTypeStndCode_4, max(case when seqnum = 5 then LoanRateTypeStndCode end) as LoanRateTypeStndCode_5, max(case when seqnum = 1 then LoanDueDate end) as LoanDueDate_1, max(case when seqnum = 2 then LoanDueDate end) as LoanDueDate_2, max(case when seqnum = 3 then LoanDueDate end) as LoanDueDate_3, max(case when seqnum = 4 then LoanDueDate end) as LoanDueDate_4, max(case when seqnum = 5 then LoanDueDate end) as LoanDueDate_5, max(case when seqnum = 1 then LoanTermMonths end) as LoanTermMonths_1, max(case when seqnum = 2 then LoanTermMonths end) as LoanTermMonths_2, max(case when seqnum = 3 then LoanTermMonths end) as LoanTermMonths_3, max(case when seqnum = 4 then LoanTermMonths end) as LoanTermMonths_4, max(case when seqnum = 5 then LoanTermMonths end) as LoanTermMonths_5, max(case when seqnum = 1 then LoanTermYears end) as LoanTermYears_1, max(case when seqnum = 2 then LoanTermYears end) as LoanTermYears_2, max(case when seqnum = 3 then LoanTermYears end) as LoanTermYears_3, max(case when seqnum = 4 then LoanTermYears end) as LoanTermYears_4, max(case when seqnum = 5 then LoanTermYears end) as LoanTermYears_5, max(case when seqnum = 1 then InitialInterestRate end) as InitialInterestRate_1, max(case when seqnum = 2 then InitialInterestRate end) as InitialInterestRate_2, max(case when seqnum = 3 then InitialInterestRate end) as InitialInterestRate_3, max(case when seqnum = 4 then InitialInterestRate end) as InitialInterestRate_4, max(case when seqnum = 5 then InitialInterestRate end) as InitialInterestRate_5, max(case when seqnum = 1 then ARMFirstAdjustmentDate end) as ARMFirstAdjustmentDate_1, max(case when seqnum = 2 then ARMFirstAdjustmentDate end) as ARMFirstAdjustmentDate_2, max(case when seqnum = 3 then ARMFirstAdjustmentDate end) as ARMFirstAdjustmentDate_3, max(case when seqnum = 4 then ARMFirstAdjustmentDate end) as ARMFirstAdjustmentDate_4, max(case when seqnum = 5 then ARMFirstAdjustmentDate end) as ARMFirstAdjustmentDate_5, max(case when seqnum = 1 then ARMFirstAdjustmentMaxRate end) as ARMFirstAdjustmentMaxRate_1, max(case when seqnum = 2 then ARMFirstAdjustmentMaxRate end) as ARMFirstAdjustmentMaxRate_2, max(case when seqnum = 3 then ARMFirstAdjustmentMaxRate end) as ARMFirstAdjustmentMaxRate_3, max(case when seqnum = 4 then ARMFirstAdjustmentMaxRate end) as ARMFirstAdjustmentMaxRate_4, max(case when seqnum = 5 then ARMFirstAdjustmentMaxRate end) as ARMFirstAdjustmentMaxRate_5, max(case when seqnum = 1 then ARMFirstAdjustmentMinRate end) as ARMFirstAdjustmentMinRate_1, max(case when seqnum = 2 then ARMFirstAdjustmentMinRate end) as ARMFirstAdjustmentMinRate_2, max(case when seqnum = 3 then ARMFirstAdjustmentMinRate end) as ARMFirstAdjustmentMinRate_3, max(case when seqnum = 4 then ARMFirstAdjustmentMinRate end) as ARMFirstAdjustmentMinRate_4, max(case when seqnum = 5 then ARMFirstAdjustmentMinRate end) as ARMFirstAdjustmentMinRate_5, max(case when seqnum = 1 then ARMIndexStndCode end) as ARMIndexStndCode_1, max(case when seqnum = 2 then ARMIndexStndCode end) as ARMIndexStndCode_2, max(case when seqnum = 3 then ARMIndexStndCode end) as ARMIndexStndCode_3, max(case when seqnum = 4 then ARMIndexStndCode end) as ARMIndexStndCode_4, max(case when seqnum = 5 then ARMIndexStndCode end) as ARMIndexStndCode_5, max(case when seqnum = 1 then ARMAdjustmentFrequencyStndCode end) as ARMAdjustmentFrequencyStndCode_1, max(case when seqnum = 2 then ARMAdjustmentFrequencyStndCode end) as ARMAdjustmentFrequencyStndCode_2, max(case when seqnum = 3 then ARMAdjustmentFrequencyStndCode end) as ARMAdjustmentFrequencyStndCode_3, max(case when seqnum = 4 then ARMAdjustmentFrequencyStndCode end) as ARMAdjustmentFrequencyStndCode_4, max(case when seqnum = 5 then ARMAdjustmentFrequencyStndCode end) as ARMAdjustmentFrequencyStndCode_5, max(case when seqnum = 1 then ARMMargin end) as ARMMargin_1, max(case when seqnum = 2 then ARMMargin end) as ARMMargin_2, max(case when seqnum = 3 then ARMMargin end) as ARMMargin_3, max(case when seqnum = 4 then ARMMargin end) as ARMMargin_4, max(case when seqnum = 5 then ARMMargin end) as ARMMargin_5, max(case when seqnum = 1 then ARMInitialCap end) as ARMInitialCap_1, max(case when seqnum = 2 then ARMInitialCap end) as ARMInitialCap_2, max(case when seqnum = 3 then ARMInitialCap end) as ARMInitialCap_3, max(case when seqnum = 4 then ARMInitialCap end) as ARMInitialCap_4, max(case when seqnum = 5 then ARMInitialCap end) as ARMInitialCap_5, max(case when seqnum = 1 then ARMPeriodicCap end) as ARMPeriodicCap_1, max(case when seqnum = 2 then ARMPeriodicCap end) as ARMPeriodicCap_2, max(case when seqnum = 3 then ARMPeriodicCap end) as ARMPeriodicCap_3, max(case when seqnum = 4 then ARMPeriodicCap end) as ARMPeriodicCap_4, max(case when seqnum = 5 then ARMPeriodicCap end) as ARMPeriodicCap_5, max(case when seqnum = 1 then ARMLifetimeCap end) as ARMLifetimeCap_1, max(case when seqnum = 2 then ARMLifetimeCap end) as ARMLifetimeCap_2, max(case when seqnum = 3 then ARMLifetimeCap end) as ARMLifetimeCap_3, max(case when seqnum = 4 then ARMLifetimeCap end) as ARMLifetimeCap_4, max(case when seqnum = 5 then ARMLifetimeCap end) as ARMLifetimeCap_5, max(case when seqnum = 1 then ARMMaxInterestRate end) as ARMMaxInterestRate_1, max(case when seqnum = 2 then ARMMaxInterestRate end) as ARMMaxInterestRate_2, max(case when seqnum = 3 then ARMMaxInterestRate end) as ARMMaxInterestRate_3, max(case when seqnum = 4 then ARMMaxInterestRate end) as ARMMaxInterestRate_4, max(case when seqnum = 5 then ARMMaxInterestRate end) as ARMMaxInterestRate_5, max(case when seqnum = 1 then ARMMinInterestRate end) as ARMMinInterestRate_1, max(case when seqnum = 2 then ARMMinInterestRate end) as ARMMinInterestRate_2, max(case when seqnum = 3 then ARMMinInterestRate end) as ARMMinInterestRate_3, max(case when seqnum = 4 then ARMMinInterestRate end) as ARMMinInterestRate_4, max(case when seqnum = 5 then ARMMinInterestRate end) as ARMMinInterestRate_5, max(case when seqnum = 1 then InterestOnlyFlag end) as InterestOnlyFlag_1, max(case when seqnum = 2 then InterestOnlyFlag end) as InterestOnlyFlag_2, max(case when seqnum = 3 then InterestOnlyFlag end) as InterestOnlyFlag_3, max(case when seqnum = 4 then InterestOnlyFlag end) as InterestOnlyFlag_4, max(case when seqnum = 5 then InterestOnlyFlag end) as InterestOnlyFlag_5, max(case when seqnum = 1 then InterestOnlyTerm end) as InterestOnlyTerm_1, max(case when seqnum = 2 then InterestOnlyTerm end) as InterestOnlyTerm_2, max(case when seqnum = 3 then InterestOnlyTerm end) as InterestOnlyTerm_3, max(case when seqnum = 4 then InterestOnlyTerm end) as InterestOnlyTerm_4, max(case when seqnum = 5 then InterestOnlyTerm end) as InterestOnlyTerm_5, max(case when seqnum = 1 then PrepaymentPenaltyFlag end) as PrepaymentPenaltyFlag_1, max(case when seqnum = 2 then PrepaymentPenaltyFlag end) as PrepaymentPenaltyFlag_2, max(case when seqnum = 3 then PrepaymentPenaltyFlag end) as PrepaymentPenaltyFlag_3, max(case when seqnum = 4 then PrepaymentPenaltyFlag end) as PrepaymentPenaltyFlag_4, max(case when seqnum = 5 then PrepaymentPenaltyFlag end) as PrepaymentPenaltyFlag_5, max(case when seqnum = 1 then PrepaymentPenaltyTerm end) as PrepaymentPenaltyTerm_1, max(case when seqnum = 2 then PrepaymentPenaltyTerm end) as PrepaymentPenaltyTerm_2, max(case when seqnum = 3 then PrepaymentPenaltyTerm end) as PrepaymentPenaltyTerm_3, max(case when seqnum = 4 then PrepaymentPenaltyTerm end) as PrepaymentPenaltyTerm_4, max(case when seqnum = 5 then PrepaymentPenaltyTerm end) as PrepaymentPenaltyTerm_5, max(case when seqnum = 1 then BiWeeklyPaymentFlag end) as BiWeeklyPaymentFlag_1, max(case when seqnum = 2 then BiWeeklyPaymentFlag end) as BiWeeklyPaymentFlag_2, max(case when seqnum = 3 then BiWeeklyPaymentFlag end) as BiWeeklyPaymentFlag_3, max(case when seqnum = 4 then BiWeeklyPaymentFlag end) as BiWeeklyPaymentFlag_4, max(case when seqnum = 5 then BiWeeklyPaymentFlag end) as BiWeeklyPaymentFlag_5, max(case when seqnum = 1 then AssumabilityRiderFlag end) as AssumabilityRiderFlag_1, max(case when seqnum = 2 then AssumabilityRiderFlag end) as AssumabilityRiderFlag_2, max(case when seqnum = 3 then AssumabilityRiderFlag end) as AssumabilityRiderFlag_3, max(case when seqnum = 4 then AssumabilityRiderFlag end) as AssumabilityRiderFlag_4, max(case when seqnum = 5 then AssumabilityRiderFlag end) as AssumabilityRiderFlag_5, max(case when seqnum = 1 then BalloonRiderFlag end) as BalloonRiderFlag_1, max(case when seqnum = 2 then BalloonRiderFlag end) as BalloonRiderFlag_2, max(case when seqnum = 3 then BalloonRiderFlag end) as BalloonRiderFlag_3, max(case when seqnum = 4 then BalloonRiderFlag end) as BalloonRiderFlag_4, max(case when seqnum = 5 then BalloonRiderFlag end) as BalloonRiderFlag_5, max(case when seqnum = 1 then CondominiumRiderFlag end) as CondominiumRiderFlag_1, max(case when seqnum = 2 then CondominiumRiderFlag end) as CondominiumRiderFlag_2, max(case when seqnum = 3 then CondominiumRiderFlag end) as CondominiumRiderFlag_3, max(case when seqnum = 4 then CondominiumRiderFlag end) as CondominiumRiderFlag_4, max(case when seqnum = 5 then CondominiumRiderFlag end) as CondominiumRiderFlag_5, max(case when seqnum = 1 then PlannedUnitDevelopmentRiderFlag end) as PlannedUnitDevelopmentRiderFlag_1, max(case when seqnum = 2 then PlannedUnitDevelopmentRiderFlag end) as PlannedUnitDevelopmentRiderFlag_2, max(case when seqnum = 3 then PlannedUnitDevelopmentRiderFlag end) as PlannedUnitDevelopmentRiderFlag_3, max(case when seqnum = 4 then PlannedUnitDevelopmentRiderFlag end) as PlannedUnitDevelopmentRiderFlag_4, max(case when seqnum = 5 then PlannedUnitDevelopmentRiderFlag end) as PlannedUnitDevelopmentRiderFlag_5, max(case when seqnum = 1 then SecondHomeRiderFlag end) as SecondHomeRiderFlag_1, max(case when seqnum = 2 then SecondHomeRiderFlag end) as SecondHomeRiderFlag_2, max(case when seqnum = 3 then SecondHomeRiderFlag end) as SecondHomeRiderFlag_3, max(case when seqnum = 4 then SecondHomeRiderFlag end) as SecondHomeRiderFlag_4, max(case when seqnum = 5 then SecondHomeRiderFlag end) as SecondHomeRiderFlag_5, max(case when seqnum = 1 then OneToFourFamilyRiderFlag end) as OneToFourFamilyRiderFlag_1, max(case when seqnum = 2 then OneToFourFamilyRiderFlag end) as OneToFourFamilyRiderFlag_2, max(case when seqnum = 3 then OneToFourFamilyRiderFlag end) as OneToFourFamilyRiderFlag_3, max(case when seqnum = 4 then OneToFourFamilyRiderFlag end) as OneToFourFamilyRiderFlag_4, max(case when seqnum = 5 then OneToFourFamilyRiderFlag end) as OneToFourFamilyRiderFlag_5, max(case when seqnum = 1 then ConcurrentMtgeDocOrBkPg end) as ConcurrentMtgeDocOrBkPg_1, max(case when seqnum = 2 then ConcurrentMtgeDocOrBkPg end) as ConcurrentMtgeDocOrBkPg_2, max(case when seqnum = 3 then ConcurrentMtgeDocOrBkPg end) as ConcurrentMtgeDocOrBkPg_3, max(case when seqnum = 4 then ConcurrentMtgeDocOrBkPg end) as ConcurrentMtgeDocOrBkPg_4, max(case when seqnum = 5 then ConcurrentMtgeDocOrBkPg end) as ConcurrentMtgeDocOrBkPg_5, max(case when seqnum = 1 then LoanNumber end) as LoanNumber_1, max(case when seqnum = 2 then LoanNumber end) as LoanNumber_2, max(case when seqnum = 3 then LoanNumber end) as LoanNumber_3, max(case when seqnum = 4 then LoanNumber end) as LoanNumber_4, max(case when seqnum = 5 then LoanNumber end) as LoanNumber_5, max(case when seqnum = 1 then MERSMINNumber end) as MERSMINNumber_1, max(case when seqnum = 2 then MERSMINNumber end) as MERSMINNumber_2, max(case when seqnum = 3 then MERSMINNumber end) as MERSMINNumber_3, max(case when seqnum = 4 then MERSMINNumber end) as MERSMINNumber_4, max(case when seqnum = 5 then MERSMINNumber end) as MERSMINNumber_5, max(case when seqnum = 1 then CaseNumber end) as CaseNumber_1, max(case when seqnum = 2 then CaseNumber end) as CaseNumber_2, max(case when seqnum = 3 then CaseNumber end) as CaseNumber_3, max(case when seqnum = 4 then CaseNumber end) as CaseNumber_4, max(case when seqnum = 5 then CaseNumber end) as CaseNumber_5, max(case when seqnum = 1 then MERSFlag end) as MERSFlag_1, max(case when seqnum = 2 then MERSFlag end) as MERSFlag_2, max(case when seqnum = 3 then MERSFlag end) as MERSFlag_3, max(case when seqnum = 4 then MERSFlag end) as MERSFlag_4, max(case when seqnum = 5 then MERSFlag end) as MERSFlag_5, max(case when seqnum = 1 then TitleCompanyName end) as TitleCompanyName_1, max(case when seqnum = 2 then TitleCompanyName end) as TitleCompanyName_2, max(case when seqnum = 3 then TitleCompanyName end) as TitleCompanyName_3, max(case when seqnum = 4 then TitleCompanyName end) as TitleCompanyName_4, max(case when seqnum = 5 then TitleCompanyName end) as TitleCompanyName_5, max(case when seqnum = 1 then TitleCompanyIDStndCode end) as TitleCompanyIDStndCode_1, max(case when seqnum = 2 then TitleCompanyIDStndCode end) as TitleCompanyIDStndCode_2, max(case when seqnum = 3 then TitleCompanyIDStndCode end) as TitleCompanyIDStndCode_3, max(case when seqnum = 4 then TitleCompanyIDStndCode end) as TitleCompanyIDStndCode_4, max(case when seqnum = 5 then TitleCompanyIDStndCode end) as TitleCompanyIDStndCode_5, max(case when seqnum = 1 then AccommodationRecordingFlag end) as AccommodationRecordingFlag_1, max(case when seqnum = 2 then AccommodationRecordingFlag end) as AccommodationRecordingFlag_2, max(case when seqnum = 3 then AccommodationRecordingFlag end) as AccommodationRecordingFlag_3, max(case when seqnum = 4 then AccommodationRecordingFlag end) as AccommodationRecordingFlag_4, max(case when seqnum = 5 then AccommodationRecordingFlag end) as AccommodationRecordingFlag_5, max(case when seqnum = 1 then UnpaidBalance end) as UnpaidBalance_1, max(case when seqnum = 2 then UnpaidBalance end) as UnpaidBalance_2, max(case when seqnum = 3 then UnpaidBalance end) as UnpaidBalance_3, max(case when seqnum = 4 then UnpaidBalance end) as UnpaidBalance_4, max(case when seqnum = 5 then UnpaidBalance end) as UnpaidBalance_5, max(case when seqnum = 1 then InstallmentAmount end) as InstallmentAmount_1, max(case when seqnum = 2 then InstallmentAmount end) as InstallmentAmount_2, max(case when seqnum = 3 then InstallmentAmount end) as InstallmentAmount_3, max(case when seqnum = 4 then InstallmentAmount end) as InstallmentAmount_4, max(case when seqnum = 5 then InstallmentAmount end) as InstallmentAmount_5, max(case when seqnum = 1 then InstallmentDueDate end) as InstallmentDueDate_1, max(case when seqnum = 2 then InstallmentDueDate end) as InstallmentDueDate_2, max(case when seqnum = 3 then InstallmentDueDate end) as InstallmentDueDate_3, max(case when seqnum = 4 then InstallmentDueDate end) as InstallmentDueDate_4, max(case when seqnum = 5 then InstallmentDueDate end) as InstallmentDueDate_5, max(case when seqnum = 1 then TotalDelinquentAmount end) as TotalDelinquentAmount_1, max(case when seqnum = 2 then TotalDelinquentAmount end) as TotalDelinquentAmount_2, max(case when seqnum = 3 then TotalDelinquentAmount end) as TotalDelinquentAmount_3, max(case when seqnum = 4 then TotalDelinquentAmount end) as TotalDelinquentAmount_4, max(case when seqnum = 5 then TotalDelinquentAmount end) as TotalDelinquentAmount_5, max(case when seqnum = 1 then DelinquentAsOfDate end) as DelinquentAsOfDate_1, max(case when seqnum = 2 then DelinquentAsOfDate end) as DelinquentAsOfDate_2, max(case when seqnum = 3 then DelinquentAsOfDate end) as DelinquentAsOfDate_3, max(case when seqnum = 4 then DelinquentAsOfDate end) as DelinquentAsOfDate_4, max(case when seqnum = 5 then DelinquentAsOfDate end) as DelinquentAsOfDate_5, max(case when seqnum = 1 then CurrentLender end) as CurrentLender_1, max(case when seqnum = 2 then CurrentLender end) as CurrentLender_2, max(case when seqnum = 3 then CurrentLender end) as CurrentLender_3, max(case when seqnum = 4 then CurrentLender end) as CurrentLender_4, max(case when seqnum = 5 then CurrentLender end) as CurrentLender_5, max(case when seqnum = 1 then CurrentLenderTypeStndCode end) as CurrentLenderTypeStndCode_1, max(case when seqnum = 2 then CurrentLenderTypeStndCode end) as CurrentLenderTypeStndCode_2, max(case when seqnum = 3 then CurrentLenderTypeStndCode end) as CurrentLenderTypeStndCode_3, max(case when seqnum = 4 then CurrentLenderTypeStndCode end) as CurrentLenderTypeStndCode_4, max(case when seqnum = 5 then CurrentLenderTypeStndCode end) as CurrentLenderTypeStndCode_5, max(case when seqnum = 1 then CurrentLenderIDStndCode end) as CurrentLenderIDStndCode_1, max(case when seqnum = 2 then CurrentLenderIDStndCode end) as CurrentLenderIDStndCode_2, max(case when seqnum = 3 then CurrentLenderIDStndCode end) as CurrentLenderIDStndCode_3, max(case when seqnum = 4 then CurrentLenderIDStndCode end) as CurrentLenderIDStndCode_4, max(case when seqnum = 5 then CurrentLenderIDStndCode end) as CurrentLenderIDStndCode_5, max(case when seqnum = 1 then TrusteeSaleNumber end) as TrusteeSaleNumber_1, max(case when seqnum = 2 then TrusteeSaleNumber end) as TrusteeSaleNumber_2, max(case when seqnum = 3 then TrusteeSaleNumber end) as TrusteeSaleNumber_3, max(case when seqnum = 4 then TrusteeSaleNumber end) as TrusteeSaleNumber_4, max(case when seqnum = 5 then TrusteeSaleNumber end) as TrusteeSaleNumber_5, max(case when seqnum = 1 then AttorneyFileNumber end) as AttorneyFileNumber_1, max(case when seqnum = 2 then AttorneyFileNumber end) as AttorneyFileNumber_2, max(case when seqnum = 3 then AttorneyFileNumber end) as AttorneyFileNumber_3, max(case when seqnum = 4 then AttorneyFileNumber end) as AttorneyFileNumber_4, max(case when seqnum = 5 then AttorneyFileNumber end) as AttorneyFileNumber_5, max(case when seqnum = 1 then AuctionDate end) as AuctionDate_1, max(case when seqnum = 2 then AuctionDate end) as AuctionDate_2, max(case when seqnum = 3 then AuctionDate end) as AuctionDate_3, max(case when seqnum = 4 then AuctionDate end) as AuctionDate_4, max(case when seqnum = 5 then AuctionDate end) as AuctionDate_5, max(case when seqnum = 1 then AuctionTime end) as AuctionTime_1, max(case when seqnum = 2 then AuctionTime end) as AuctionTime_2, max(case when seqnum = 3 then AuctionTime end) as AuctionTime_3, max(case when seqnum = 4 then AuctionTime end) as AuctionTime_4, max(case when seqnum = 5 then AuctionTime end) as AuctionTime_5, max(case when seqnum = 1 then AuctionFullStreetAddress end) as AuctionFullStreetAddress_1, max(case when seqnum = 2 then AuctionFullStreetAddress end) as AuctionFullStreetAddress_2, max(case when seqnum = 3 then AuctionFullStreetAddress end) as AuctionFullStreetAddress_3, max(case when seqnum = 4 then AuctionFullStreetAddress end) as AuctionFullStreetAddress_4, max(case when seqnum = 5 then AuctionFullStreetAddress end) as AuctionFullStreetAddress_5, max(case when seqnum = 1 then AuctionCityName end) as AuctionCityName_1, max(case when seqnum = 2 then AuctionCityName end) as AuctionCityName_2, max(case when seqnum = 3 then AuctionCityName end) as AuctionCityName_3, max(case when seqnum = 4 then AuctionCityName end) as AuctionCityName_4, max(case when seqnum = 5 then AuctionCityName end) as AuctionCityName_5, max(case when seqnum = 1 then StartingBid end) as StartingBid_1, max(case when seqnum = 2 then StartingBid end) as StartingBid_2, max(case when seqnum = 3 then StartingBid end) as StartingBid_3, max(case when seqnum = 4 then StartingBid end) as StartingBid_4, max(case when seqnum = 5 then StartingBid end) as StartingBid_5, max(case when seqnum = 1 then KeyedDate end) as KeyedDate_1, max(case when seqnum = 2 then KeyedDate end) as KeyedDate_2, max(case when seqnum = 3 then KeyedDate end) as KeyedDate_3, max(case when seqnum = 4 then KeyedDate end) as KeyedDate_4, max(case when seqnum = 5 then KeyedDate end) as KeyedDate_5, max(case when seqnum = 1 then KeyerID end) as KeyerID_1, max(case when seqnum = 2 then KeyerID end) as KeyerID_2, max(case when seqnum = 3 then KeyerID end) as KeyerID_3, max(case when seqnum = 4 then KeyerID end) as KeyerID_4, max(case when seqnum = 5 then KeyerID end) as KeyerID_5, max(case when seqnum = 1 then SubVendorStndCode end) as SubVendorStndCode_1, max(case when seqnum = 2 then SubVendorStndCode end) as SubVendorStndCode_2, max(case when seqnum = 3 then SubVendorStndCode end) as SubVendorStndCode_3, max(case when seqnum = 4 then SubVendorStndCode end) as SubVendorStndCode_4, max(case when seqnum = 5 then SubVendorStndCode end) as SubVendorStndCode_5, max(case when seqnum = 1 then ImageFileName end) as ImageFileName_1, max(case when seqnum = 2 then ImageFileName end) as ImageFileName_2, max(case when seqnum = 3 then ImageFileName end) as ImageFileName_3, max(case when seqnum = 4 then ImageFileName end) as ImageFileName_4, max(case when seqnum = 5 then ImageFileName end) as ImageFileName_5, max(case when seqnum = 1 then BuilderFlag end) as BuilderFlag_1, max(case when seqnum = 2 then BuilderFlag end) as BuilderFlag_2, max(case when seqnum = 3 then BuilderFlag end) as BuilderFlag_3, max(case when seqnum = 4 then BuilderFlag end) as BuilderFlag_4, max(case when seqnum = 5 then BuilderFlag end) as BuilderFlag_5, max(case when seqnum = 1 then MatchStndCode end) as MatchStndCode_1, max(case when seqnum = 2 then MatchStndCode end) as MatchStndCode_2, max(case when seqnum = 3 then MatchStndCode end) as MatchStndCode_3, max(case when seqnum = 4 then MatchStndCode end) as MatchStndCode_4, max(case when seqnum = 5 then MatchStndCode end) as MatchStndCode_5, max(case when seqnum = 1 then REOStndCode end) as REOStndCode_1, max(case when seqnum = 2 then REOStndCode end) as REOStndCode_2, max(case when seqnum = 3 then REOStndCode end) as REOStndCode_3, max(case when seqnum = 4 then REOStndCode end) as REOStndCode_4, max(case when seqnum = 5 then REOStndCode end) as REOStndCode_5, max(case when seqnum = 1 then UpdateOwnershipFlag end) as UpdateOwnershipFlag_1, max(case when seqnum = 2 then UpdateOwnershipFlag end) as UpdateOwnershipFlag_2, max(case when seqnum = 3 then UpdateOwnershipFlag end) as UpdateOwnershipFlag_3, max(case when seqnum = 4 then UpdateOwnershipFlag end) as UpdateOwnershipFlag_4, max(case when seqnum = 5 then UpdateOwnershipFlag end) as UpdateOwnershipFlag_5, max(case when seqnum = 1 then LoadID3 end) as LoadID3_1, max(case when seqnum = 2 then LoadID3 end) as LoadID3_2, max(case when seqnum = 3 then LoadID3 end) as LoadID3_3, max(case when seqnum = 4 then LoadID3 end) as LoadID3_4, max(case when seqnum = 5 then LoadID3 end) as LoadID3_5, max(case when seqnum = 1 then StatusInd end) as StatusInd_1, max(case when seqnum = 2 then StatusInd end) as StatusInd_2, max(case when seqnum = 3 then StatusInd end) as StatusInd_3, max(case when seqnum = 4 then StatusInd end) as StatusInd_4, max(case when seqnum = 5 then StatusInd end) as StatusInd_5, max(case when seqnum = 1 then TransactionTypeStndCode end) as TransactionTypeStndCode_1, max(case when seqnum = 2 then TransactionTypeStndCode end) as TransactionTypeStndCode_2, max(case when seqnum = 3 then TransactionTypeStndCode end) as TransactionTypeStndCode_3, max(case when seqnum = 4 then TransactionTypeStndCode end) as TransactionTypeStndCode_4, max(case when seqnum = 5 then TransactionTypeStndCode end) as TransactionTypeStndCode_5, max(case when seqnum = 1 then BatchID3 end) as BatchID3_1, max(case when seqnum = 2 then BatchID3 end) as BatchID3_2, max(case when seqnum = 3 then BatchID3 end) as BatchID3_3, max(case when seqnum = 4 then BatchID3 end) as BatchID3_4, max(case when seqnum = 5 then BatchID3 end) as BatchID3_5, max(case when seqnum = 1 then BKFSPID end) as BKFSPID_1, max(case when seqnum = 2 then BKFSPID end) as BKFSPID_2, max(case when seqnum = 3 then BKFSPID end) as BKFSPID_3, max(case when seqnum = 4 then BKFSPID end) as BKFSPID_4, max(case when seqnum = 5 then BKFSPID end) as BKFSPID_5, max(case when seqnum = 1 then ZVendorStndCode end) as ZVendorStndCode_1, max(case when seqnum = 2 then ZVendorStndCode end) as ZVendorStndCode_2, max(case when seqnum = 3 then ZVendorStndCode end) as ZVendorStndCode_3, max(case when seqnum = 4 then ZVendorStndCode end) as ZVendorStndCode_4, max(case when seqnum = 5 then ZVendorStndCode end) as ZVendorStndCode_5, max(case when seqnum = 1 then SourceChkSum end) as SourceChkSum_1, max(case when seqnum = 2 then SourceChkSum end) as SourceChkSum_2, max(case when seqnum = 3 then SourceChkSum end) as SourceChkSum_3, max(case when seqnum = 4 then SourceChkSum end) as SourceChkSum_4, max(case when seqnum = 5 then SourceChkSum end) as SourceChkSum_5 from (select t.*, (select count(*) from t t2 where t2.TransId = t.TransId and t2.FIPS3 <= t.FIPS3) as seqnum from t) t group by t.TransId;